***Clonación del repositorio***

In [1]:
!git clone https://github.com/ignacio-alvc/usgs-data-pipeline-gcp.git

fatal: destination path 'usgs-data-pipeline-gcp' already exists and is not an empty directory.


***Instalación de librerías***

In [2]:
!pip install xgboost google-cloud-bigquery[pandas] joblib google-cloud-storage scikit-learn pandas

***Importaciones***

In [3]:
import requests
import pandas as pd
from datetime import datetime, timedelta, timezone
import os

In [4]:
GCP_CREDENTIALS_FILENAME = "portfolio-earthquake-analysis-6ff1f9179260.json"
GCP_CREDENTIALS_PATH = f"/content/{GCP_CREDENTIALS_FILENAME}"
REPO_NAME = "usgs-data-pipeline-gcp"
REPO_PATH = f"/content/{REPO_NAME}"
MODEL_DIR = os.path.join(REPO_PATH, "models")
os.makedirs(MODEL_DIR, exist_ok=True)

print("Entorno listo. Repositorio clonado y librerías instaladas.")
print(f"Los modelos se guardaran en: {MODEL_DIR}")

Entorno listo. Repositorio clonado y librerías instaladas.
Los modelos se guardaran en: /content/usgs-data-pipeline-gcp/models


In [8]:
end_time = datetime.now(timezone.utc)
start_time = end_time - timedelta(days=180)
start_str = start_time.strftime('%Y-%m-%dT%H:%M:%S')
end_str = end_time.strftime('%Y-%m-%dT%H:%M:%S')

api_url = f"https://earthquake.usgs.gov/fdsnws/event/1/query?format=geojson&starttime={start_str}&endtime={end_str}&minmagnitude=2.5"

print(f"Descargando datos desde {start_time.date()} hasta {end_time.date()}...")

try:
    response = requests.get(api_url)
    response.raise_for_status()
    data = response.json()
    features = data.get('features', [])

    if not features:
        print("No se encontraron sismos.")
        df = pd.DataFrame()
    else:
        records = []
        for feature in features:
            props = feature.get('properties', {})
            geom = feature.get('geometry', {})
            coords = geom.get('coordinates', [None, None, None])
            if props.get('mag') is not None and len(coords) == 3 and coords[0] is not None and coords[1] is not None and coords[2] is not None:
                 records.append({
                     'magnitude': props.get('mag'),
                     'latitude': coords[1],
                     'longitude': coords[0],
                     'depth_km': coords[2],
                 })
        df = pd.DataFrame(records)
        df.dropna(inplace=True)

    print(f"Se procesaron {len(df)} sismos (mag >= 2.5).")
    df.head()

except requests.exceptions.RequestException as e:
    print(f"Error al descargar datos: {e}")
    df = pd.DataFrame()
if not os.path.exists(GCP_CREDENTIALS_PATH):
    print(f"ERROR: No se encontró el archivo de credenciales en {GCP_CREDENTIALS_PATH}. ¿Lo subiste?")
else:
    print("Archivo de credenciales encontrado.")

Descargando datos desde 2025-04-28 hasta 2025-10-25...
Se procesaron 14403 sismos (mag >= 2.5).
Archivo de credenciales encontrado.


In [9]:
# --- Título: Preprocesamiento y División de Datos ---

from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
import joblib

if not df.empty:
    # 1. Separar características (X) y objetivo (y)
    features = ['latitude', 'longitude', 'depth_km']
    target = 'magnitude'
    X = df[features]
    y = df[target]

    # 2. Dividir en entrenamiento y prueba
    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

    # 3. Escalar características
    scaler = StandardScaler()
    X_train_scaled = scaler.fit_transform(X_train)
    X_test_scaled = scaler.transform(X_test)

    # 4. Guardar el escalador
    SCALER_FILENAME = os.path.join(MODEL_DIR, "earthquake_scaler.joblib")
    joblib.dump(scaler, SCALER_FILENAME)

    print("Preprocesamiento completado. Escalador guardado.")
    print(f"Scaler: {SCALER_FILENAME}")
    # Verificar que el archivo existe
    !ls -lh {MODEL_DIR}
else:
    print("El DataFrame está vacío. No se puede continuar con el preprocesamiento.")

Preprocesamiento completado. Escalador guardado.
Scaler: /content/usgs-data-pipeline-gcp/models/earthquake_scaler.joblib
total 4.0K
-rw-r--r-- 1 root root 959 Oct 25 18:41 earthquake_scaler.joblib


In [10]:
# --- Título: Entrenamiento del Modelo (XGBoost) ---

import xgboost as xgb
from sklearn.model_selection import GridSearchCV
from sklearn.metrics import mean_squared_error
import numpy as np

if not df.empty:
    print("Iniciando entrenamiento con XGBoost y GridSearchCV...")

    xgb_reg = xgb.XGBRegressor(objective='reg:squarederror', random_state=42)

    param_grid = {
        'n_estimators': [100, 200],
        'learning_rate': [0.1, 0.05],
        'max_depth': [3, 5]
    }

    grid_search = GridSearchCV(
        estimator=xgb_reg,
        param_grid=param_grid,
        cv=5, # 5-fold cross-validation
        scoring='neg_mean_squared_error',
        verbose=1,
        n_jobs=-1
    )

    grid_search.fit(X_train_scaled, y_train)
    best_xgb_model = grid_search.best_estimator_

    print("\nMejores hiperparámetros:")
    print(grid_search.best_params_)

    # Evaluar
    y_pred = best_xgb_model.predict(X_test_scaled)
    rmse = np.sqrt(mean_squared_error(y_test, y_pred))
    print(f"\nRMSE en prueba: {rmse:.4f}")

    # Guardar modelo
    MODEL_FILENAME = os.path.join(MODEL_DIR, "earthquake_xgboost_model.joblib")
    joblib.dump(best_xgb_model, MODEL_FILENAME)
    print(f"Modelo XGBoost guardado en: {MODEL_FILENAME}")
    # Verificar que el archivo existe
    !ls -lh {MODEL_DIR}

else:
     print("El DataFrame está vacío. No se puede entrenar el modelo.")

Iniciando entrenamiento con XGBoost y GridSearchCV...
Fitting 5 folds for each of 8 candidates, totalling 40 fits

Mejores hiperparámetros:
{'learning_rate': 0.1, 'max_depth': 5, 'n_estimators': 100}

RMSE en prueba: 0.4214
Modelo XGBoost guardado en: /content/usgs-data-pipeline-gcp/models/earthquake_xgboost_model.joblib
total 256K
-rw-r--r-- 1 root root  959 Oct 25 18:41 earthquake_scaler.joblib
-rw-r--r-- 1 root root 252K Oct 25 18:42 earthquake_xgboost_model.joblib


***Definición de variables***

**Configuración**

**Carga de datos desde BigQuery**